## 기준1. 좌표로 거리 구하기       
st_end_daram_df

다람쥐 버스 정류장의 시작점과 종점의 직선거리를 구하여, 기존 다람쥐 버스 노선의 운행 범위를 알아보려고 함.

In [29]:
import sys
import pandas as pd

sys.path.append('./EDA')
sys.path.append('./Modeling')
sys.path.append('./Data/csv')

from EDA_STEP_1 import *
from visualization import *
from clustering import *

In [30]:
st_end_daram_df = get_daram_14_station_df()
st_end_daram_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 5 to 91
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ROUTE_ID           14 non-null     object 
 1   노선명                14 non-null     object 
 2   순번                 14 non-null     object 
 3   NODE_ID            14 non-null     object 
 4   ARS_ID             14 non-null     object 
 5   정류소명               14 non-null     object 
 6   X좌표                14 non-null     object 
 7   Y좌표                14 non-null     object 
 8   법정동코드              14 non-null     object 
 9   법정동_구              14 non-null     object 
 10  법정동                14 non-null     object 
 11  academy_cnt        14 non-null     int32  
 12  kindergarten_cnt   14 non-null     int32  
 13  mart_cnt           14 non-null     int32  
 14  restaurant_cnt     14 non-null     int32  
 15  school_cnt         14 non-null     int32  
 16  university_cnt     14 non-nu

In [31]:
start_station = ['111000128','113000113','120000156','120000109','105000127','122000305','123000209']
end_station =   ['111000291','118000048','119000024','120000018','105000072','122000302','123000043']
st_end_node_list = []
for st_end_node in zip(start_station,end_station):
    st_end_node_list.append(st_end_node)
st_end_node_list

[('111000128', '111000291'),
 ('113000113', '118000048'),
 ('120000156', '119000024'),
 ('120000109', '120000018'),
 ('105000127', '105000072'),
 ('122000305', '122000302'),
 ('123000209', '123000043')]

In [32]:
import math
# 하버사인 공식:
## - 지구 곡률을 반영하여 두 지표 사이의 거리를 각도, 호를 사용하여 구하는 방식.
## latitude: 위도(36), longitude: 경도(127)

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance


In [33]:
df_node_distance = pd.DataFrame(columns=['BUS_NUM','Radius'])
for bus in st_end_node_list:
    lon1 = float(st_end_daram_df['X좌표'][st_end_daram_df['NODE_ID']== bus[0]].iloc[0])
    lat1 = float(st_end_daram_df['Y좌표'][st_end_daram_df['NODE_ID']== bus[0]].iloc[0])
    lon2 = float(st_end_daram_df['X좌표'][st_end_daram_df['NODE_ID']== bus[1]].iloc[0])
    lat2 = float(st_end_daram_df['Y좌표'][st_end_daram_df['NODE_ID']== bus[1]].iloc[0])
    bus_num = st_end_daram_df['노선명'][st_end_daram_df['NODE_ID']==bus[0]].iloc[0]
    
    radius = haversine_distance(lat1, lon1, lat2, lon2)
    new_data = {'BUS_NUM':bus_num,'Radius':radius}
    df_node_distance = df_node_distance.append(new_data, ignore_index=True)

In [34]:
df_node_distance

,BUS_NUM,Radius
0,8771,2.253933
1,8761,3.270516
2,8551,3.377080
3,8552,2.189844
4,8221,2.017190
5,8441,2.383591
6,8331,4.338116


---

기준 1 : 모든 다람쥐 버스 노선의 시작점과 종점의 직선거리는 2km이상 4.5km 이내이다.

## 기준 1에 따라 후보군 제거하기

### 1-1. GMM 클러스터된 다람쥐 데이터 가져오기  
GMM 군집결과 총 11개의 클래스.

In [35]:
# gmm 다람쥐 csv 가져오기
gmm_daram_df = pd.read_csv('./Data/csv/gmm_daram.csv', encoding='UTF-8')
gmm_daram_df=gmm_daram_df[['X좌표','Y좌표','NODE_ID', 'gmm_cluster']]
gmm_daram_df

# 다람쥐 95개 정류장 csv 가져오기
daram_df = pd.read_csv('./Data/csv/다람쥐95개정류장.csv', encoding='cp949')

### 1-2.유사도로 뽑은 버스 데이터 가져오기

In [36]:
cosine_daram_df=pd.read_csv('./Data/csv/cosine_similarity_daram.csv', encoding='utf-8')

### 1-3. 95개 다람쥐 버스 정류장에 gmm 군집결과를 labeling 

In [37]:
gmm_daram_result_df = pd.merge(daram_df,gmm_daram_df, left_on='NODE_ID' ,right_on='NODE_ID',how='left')

In [38]:
gmm_daram_result_df.shape

(95, 32)

In [39]:
# 불필요한 컬럼 삭제 
gmm_daram_result_df=gmm_daram_result_df.drop(['X좌표_y','Y좌표_y'], axis=1)
gmm_daram_result_df.rename(columns={'X좌표_x':'X좌표','Y좌표_x' :'Y좌표'}, inplace=True)

### 1-4. GMM 클러스터링된 코사인 유사도에서 후보군 제거 

In [40]:
gmm_daram_result_df['구-동']=gmm_daram_result_df['법정동_구']+gmm_daram_result_df['법정동']
# 기존 다람쥐 버스 (95개)가 있는 동을 리스트로 뽑기 
daram_95_gu_list = daram_df['법정동_구'].drop_duplicates().tolist()
daram_95_gu_dong_list = daram_df['법정동'].drop_duplicates().tolist()
daram_95_gu_dong_list

['장안동',
 '답십리동',
 '신천동',
 '방이동',
 '오금동',
 '마천동',
 '수서동',
 '세곡동',
 '자곡동',
 '노량진동',
 '상도동',
 '봉천동',
 '신림동',
 '창전동',
 '노고산동',
 '여의도동',
 '신사동',
 '응암동',
 '구산동',
 '역촌동']

In [42]:
# 다람쥐 버스가 있던 동 제거
drop_cosine_daram_df = gmm_daram_result_df[~gmm_daram_result_df['구-동'].isin(daram_95_gu_dong_list)]
# 다람쥐 버스가 있던 구 제거
drop_gu_cosine_daram_df = drop_cosine_daram_df[~drop_cosine_daram_df['법정동_구'].isin(daram_95_gu_list)]

### 1-5. 기존 다람쥐 버스 정류장 근처 지역 제거       
새로운 지역에 다람쥐 버스 경로를 신설하기 위해, 기존 다람쥐 버스와 가깝지 않은 정류장들을 선택 해야함. 

In [43]:
# 동이 뭐가 있는지 보기
region_list=drop_cosine_daram_df['법정동'].drop_duplicates().tolist()
region_list

['장안동',
 '답십리동',
 '신천동',
 '방이동',
 '오금동',
 '마천동',
 '수서동',
 '세곡동',
 '자곡동',
 '노량진동',
 '상도동',
 '봉천동',
 '신림동',
 '창전동',
 '노고산동',
 '여의도동',
 '신사동',
 '응암동',
 '구산동',
 '역촌동']

In [44]:
drop_cosine_daram_df

,Unnamed: 0.1,Unnamed: 0,NODE_ID,정류소명,X좌표,Y좌표,법정동코드,법정동_구,법정동,academy_cnt,...,public_office_cnt,employee_cnt,alone_ratio,emp_corp_ratio,population_15to64,RIDE_SUM_6_10,ALIGHT_SUM_6_10,노선번호,gmm_cluster,구-동
0,0,1969,105000059,전곡시장앞,127.067795,37.577237,1123010600,동대문구,장안동,57,...,1,36170,0.127863,3.528780,48942,129894,74930,8221,NaN,동대문구장안동
1,1,1971,105000061,답십리촬영소.우성그린아파트,127.066930,37.574353,1123010500,동대문구,답십리동,58,...,0,18996,0.129167,3.160732,40211,167439,65658,8221,NaN,동대문구답십리동
2,2,1974,105000064,동답한신동아아파트,127.064217,37.569746,1123010500,동대문구,답십리동,34,...,4,18996,0.129167,3.160732,40211,285028,66170,8221,NaN,동대문구답십리동
3,3,1977,105000067,동아.위더스빌아파트,127.061465,37.566159,1123010500,동대문구,답십리동,34,...,6,18996,0.129167,3.160732,40211,175854,110733,8221,NaN,동대문구답십리동
4,4,1981,105000071,고미술상가,127.057518,37.563532,1123010600,동대문구,장안동,23,...,4,36170,0.127863,3.528780,48942,21393,211227,8221,NaN,동대문구장안동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,90,4957,111000129,시립서북병원세현교회앞,126.910460,37.606017,1138010800,은평구,역촌동,60,...,3,8796,0.122771,2.643029,32310,427373,51329,8771,NaN,은평구역촌동
91,91,4958,111000130,서울기독대학교입구,126.910227,37.602032,1138010800,은평구,역촌동,100,...,3,8796,0.122771,2.643029,32310,363785,35764,8771,NaN,은평구역촌동
92,92,4959,111000131,우남교통,126.910125,37.599767,1138010900,은평구,신사동,82,...,2,8233,0.115241,2.873647,33579,359679,35065,8771,NaN,은평구신사동
93,93,4960,111000132,신사동고개.은평지역자활센터,126.910067,37.598601,1138010900,은평구,신사동,67,...,2,8233,0.115241,2.873647,33579,268386,204551,8771,NaN,은평구신사동


In [46]:
drop_gu_cosine_daram_df

,Unnamed: 0.1,Unnamed: 0,NODE_ID,정류소명,X좌표,Y좌표,법정동코드,법정동_구,법정동,academy_cnt,...,public_office_cnt,employee_cnt,alone_ratio,emp_corp_ratio,population_15to64,RIDE_SUM_6_10,ALIGHT_SUM_6_10,노선번호,gmm_cluster,구-동


In [ ]:
## 기준1: 기존 다람버스와 같은 구, 같은 동에 있는 애들 제거했다. 로 수정하자.

# 기준2: 기점, 종점 인근에 지하철역이 적어도 하나 이상은 있어야 한다.

-> 전체 테이블에서 얘네 merge하고 subway가 0인 애들 제외하자.

In [68]:
df = pd.read_csv('./Data/csv/선정기준1_정류장.csv')
df

,Unnamed: 0.1,index,Unnamed: 0,NODE_ID,정류소명,X좌표,Y좌표,법정동코드,법정동_구,법정동,...,public_office_cnt,employee_cnt,alone_ratio,emp_corp_ratio,population_15to64,RIDE_SUM_6_10,ALIGHT_SUM_6_10,Label,gmm_cluster,구-동
0,272,5194,5194,111001114,MBC,126.890895,37.581652,1144012700,마포구,상암동,...,5,49456,0.125419,12.327019,21507,53563,17169,0,8,마포구상암동
1,273,5195,5195,111001115,MBC,126.890950,37.581894,1144012700,마포구,상암동,...,4,49456,0.125419,12.327019,21507,2630,135491,0,8,마포구상암동
2,387,5994,5994,113000075,상암초등학교,126.893281,37.576474,1144012700,마포구,상암동,...,2,49456,0.125419,12.327019,21507,168351,60031,0,8,마포구상암동
3,388,5995,5995,113000076,상암초등학교,126.893493,37.576634,1144012700,마포구,상암동,...,2,49456,0.125419,12.327019,21507,37355,260329,0,8,마포구상암동
4,393,6095,6095,113000192,상암DMC홍보관.YTN,126.891323,37.577985,1144012700,마포구,상암동,...,2,49456,0.125419,12.327019,21507,46193,487784,0,8,마포구상암동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,1310,12496,12496,124900086,천호동로데오거리,127.127198,37.540565,1174010900,강동구,천호동,...,3,29406,0.181504,3.280455,62587,1220,73802,0,5,강동구천호동
209,1311,12497,12497,124900087,천호우체국,127.125362,37.540860,1174010900,강동구,천호동,...,4,29406,0.181504,3.280455,62587,18355,3539,0,5,강동구천호동
210,1312,12498,12498,124900088,천호동로데오거리,127.127189,37.540362,1174010900,강동구,천호동,...,3,29406,0.181504,3.280455,62587,6823,5343,0,5,강동구천호동
211,1313,12499,12499,124900089,암사종합시장,127.128250,37.549699,1174010900,강동구,천호동,...,4,29406,0.181504,3.280455,62587,36479,1697,0,7,강동구천호동


In [69]:
get_clustering_folium(df, "X좌표", "Y좌표", label_column='gmm_cluster')

In [72]:
# subway >0 이상인 애들만 걸러내기
df_sub = df[df['subway_cnt']>0]
get_clustering_folium(df_sub, "X좌표", "Y좌표", label_column='gmm_cluster')